# THOMAS NGO
# TEVIN VU
# MICHAEL DANG

In [1]:
import pandas as pd
import numpy as np
import heapq
import pprint
import random

from sklearn.model_selection import train_test_split
from tqdm import tqdm
from collections import defaultdict


### 1. Use the contents of ratings.csv to create a dataset for your network. There should be a feature vector for each user, with each feature corresponding to a movie. Encode movies that the user has rated 3.0 or above as +1, and other movies as -1.

In [134]:
!wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2021-04-28 01:55:52--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  1.53MB/s    in 0.6s    

2021-04-28 01:55:53 (1.53 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [2]:
!unzip -n ml-latest-small.zip

Archive:  ml-latest-small.zip


In [136]:
!ls

ml-latest-small  ml-latest-small.zip  sample_data


In [3]:
rating_df = pd.read_csv('ml-latest-small/ratings.csv')
rating_df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [4]:
rating_df['encode'] = np.where(rating_df['rating'] >= 3.0, 1, -1)

In [5]:
rating_df.head()

,userId,movieId,rating,timestamp,encode
0,1,1,4.0,964982703,1
1,1,3,4.0,964981247,1
2,1,6,4.0,964982224,1
3,1,47,5.0,964983815,1
4,1,50,5.0,964982931,1


In [6]:
rating_df[rating_df['rating'] < 3.0]

,userId,movieId,rating,timestamp,encode
76,1,1219,2.0,964983393,-1
143,1,2253,2.0,964981775,-1
148,1,2338,2.0,964983546,-1
152,1,2389,2.0,964983094,-1
170,1,2617,2.0,964982588,-1
...,...,...,...,...,...
100766,610,135536,2.5,1493847638,-1
100772,610,136598,2.5,1493847366,-1
100778,610,138610,1.5,1493848976,-1
100797,610,145951,0.5,1493849019,-1


In [114]:
print(f'total number of users: {len(set(rating_df.userId))}')

total number of users: 610


In [7]:
movies_df = pd.read_csv('ml-latest-small/movies.csv')
col_names = movies_df['movieId'].values.reshape(-1,)
num_row = rating_df['userId'].values[-1]
dataset_df = pd.DataFrame(index = range(0, num_row), columns = col_names) # create dataset, each column corresponding to a movieId
dataset_df = dataset_df.fillna(-1)
  
for i in tqdm(range(num_row), desc='Creating dataset', leave=True, position=0):
    tmp_df = rating_df[rating_df['userId'] == i+1]
    for _, row in tmp_df.iterrows():
        dataset_df.at[i, row['movieId']] = row['encode'] 

Creating dataset: 100%|██████████| 610/610 [00:10<00:00, 58.02it/s]


In [72]:
dataset_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
606,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
607,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,-1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
608,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [9]:
dataset_df.iloc[0].head()

1    1
2   -1
3    1
4   -1
5   -1
Name: 0, dtype: int64

In [10]:
entire_dataset = dataset_df.values
entire_dataset

array([[ 1, -1,  1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [ 1, -1, -1, ..., -1, -1, -1],
       [ 1, -1, -1, ..., -1, -1, -1]])

In [11]:
entire_dataset[0]

array([ 1, -1,  1, ..., -1, -1, -1])

### 2. Set aside 10% of your dataset for testing. How many users are there in the training set? What is the storage capacity of the network? Is the network likely to be able to store the dataset?

In [115]:
train_set, test_set = train_test_split(entire_dataset, test_size=0.10, shuffle=True)

In [71]:
train_set.shape

(549, 9742)

In [116]:
train_set[0]

array([ 1, -1, -1, ..., -1, -1, -1])

In [117]:
test_set.shape

(61, 9742)

In [118]:
train_set[0]

array([ 1, -1, -1, ..., -1, -1, -1])

#### How many users are there in the training set?
- There are 549 users

#### What is the storage capacity of the network?

In [119]:
d = len(dataset_df.columns) # number of features/neurons
print(f'd: {d}')
# the size of training data that a Hopfield network with d visible units can store without causing errors
# in associative recall. This is known as storage capacity of a Hopfield network (p. 240)
print(f'The storage capacity: {round(0.15 * d)} training examples')

d: 9742
The storage capacity: 1461 training examples


#### Is the network likely to be able to store the dataset?

In [120]:
print('The total number of bits that Hopfield network can store: ')
print(f'{round(0.15 * d * d)} bits')

The total number of bits that Hopfield network can store: 
14235985 bits


In [121]:
print('The number of weights in the network is:')
print((d*(d-1))/2)

The number of weights in the network is:
47448411.0


In [122]:
print('The dataset contains: ')
print(f'{610 * 9742} bits')

The dataset contains: 
5942620 bits


Yes, the network is likely to able to store the dataset because the storage capacity can contain 1461 examples in which can be memorized from a Hopfield network. Furthermore, there are only 610 examples in our dataset.

### 3. Construct a Hopfield network for your dataset and train it on your training set.
- Hint: You may want to take a look at the pseudocode on pp. 66-67 of Artificial Intelligence Engines.
- What accuracy does your trained network achieve?


In [123]:
def learning(data):
    row, col = data.shape
    LEARNING_RATE = 1/row
    
    # initialize the weights W to zero
    W = np.zeros(shape=(col, col))
    b = np.zeros(col)
    
    for x in tqdm(data, desc='learning', leave=True, position=0):
        # find weights for the vector x_t
        W_t = LEARNING_RATE * np.outer(x, x)
        # update weights and bias
        W += W_t
        b += x
    W = np.vstack((b, W))
    b = np.insert(b, 0, 0)
    W = np.hstack((b.reshape(-1,1), W))
    np.fill_diagonal(W, 0)
    return W

In [124]:
W = learning(train_set)
print(W)

learning: 100%|██████████| 549/549 [03:24<00:00,  2.69it/s]


[[ 0.00000000e+00 -2.07000000e+02 -3.99000000e+02 ... -5.47000000e+02
  -5.47000000e+02 -5.47000000e+02]
 [-2.07000000e+02  0.00000000e+00  4.02550091e-01 ...  3.73406193e-01
   3.73406193e-01  3.73406193e-01]
 [-3.99000000e+02  4.02550091e-01  0.00000000e+00 ...  7.23132969e-01
   7.23132969e-01  7.23132969e-01]
 ...
 [-5.47000000e+02  3.73406193e-01  7.23132969e-01 ...  0.00000000e+00
   1.00000000e+00  9.92714026e-01]
 [-5.47000000e+02  3.73406193e-01  7.23132969e-01 ...  1.00000000e+00
   0.00000000e+00  9.92714026e-01]
 [-5.47000000e+02  3.73406193e-01  7.23132969e-01 ...  9.92714026e-01
   9.92714026e-01  0.00000000e+00]]


In [125]:
W.shape

(9743, 9743)

In [145]:
def recall(W, corrupted_x):
    state_y = np.copy(corrupted_x)
    state_y = np.insert(state_y, 0, -1) # add bias
    stable = False
    K = len(state_y)
    rng = np.random.default_rng()
    while not stable:
        stable = True
        indices = list(range(K))
        for k in range(K):
            j = np.random.randint(K)
            u_j = np.dot(W[j], state_y)
            state_y_last = state_y[j]
            state_y[j] = (1 if u_j > 0 else -1)
            if state_y[j] != state_y_last:
                stable = False
    state_y = np.delete(state_y, 0)
    return state_y

In [146]:
print(f'indices of the unit state having value of 1 in input:\n{np.where(train_set[0] >= 0)}')

indices of the unit state having value of 1 in input:
(array([   0,    4,    5,    6,   16,   24,   31,   83,   84,   99,  116,
        333,  398,  412,  419,  431,  510,  520,  546,  551,  558,  577,
        592,  594,  607,  615,  621,  630,  633,  658,  809,  815, 1044]),)


In [147]:
pred = recall(W, train_set[0])
print(train_set[0].shape)
print(pred.shape)
print(f'train_set[0]: {train_set[0]}')
print(f'pred:\n{pred}')
print('\n')
print(f'indices of the unit state having value of 1 in input:\n{np.where(train_set[0] >= 0)}')
print('\n')
print(f'indices of the unit state having value of 1 in prediction:\n{np.where(pred >= 0)}')
print('\n')
print(f'is the output similar to the input?\n{np.array_equal(train_set[0], pred)}')

(9742,)
(9742,)
train_set[0]: [ 1 -1 -1 ... -1 -1 -1]
pred:
[-1 -1 -1 ... -1 -1 -1]


indices of the unit state having value of 1 in input:
(array([   0,    4,    5,    6,   16,   24,   31,   83,   84,   99,  116,
        333,  398,  412,  419,  431,  510,  520,  546,  551,  558,  577,
        592,  594,  607,  615,  621,  630,  633,  658,  809,  815, 1044]),)


indices of the unit state having value of 1 in prediction:
(array([ 99, 116, 277, 314, 419]),)


is the output similar to the input?
False


In [182]:
corrupted_x = np.copy(train_set[0])
print(f'train_set[0]: {train_set[0]}')
corrupted_x[3] = -1
print(f'corrupted_x: {corrupted_x}')
pred = recall(W, corrupted_x)
print(f'pred: {pred}')
print(f'is the output similar to the input?\n{np.array_equal(train_set[0], pred)}')
print(f'indices of the unit state having value of 1 in prediction:\n{np.where(pred >= 0)}')

train_set[0]: [ 1 -1 -1 ... -1 -1 -1]
corrupted_x: [ 1 -1 -1 ... -1 -1 -1]
pred: [-1 -1 -1 ... -1 -1 -1]
is the output similar to the input?
False
indices of the unit state having value of 1 in prediction:
(array([277, 314]),)


In [171]:
def predict(dataset):
    correct = 0
    higher_performing = defaultdict()
    new_correct = 0
    old_correct = 0
    i = 0
    for x in tqdm(dataset, desc='Predicting', leave=True, position=0):
        pred = recall(W, x)
        n = len(pred)
        for j in range(n):
            if x[j] == pred[j]:
                correct += 1
        if old_correct < correct:
            old_correct = new_correct
            higher_performing[i] = x
            i += 1
            if i == 3:
                i = 0
    total_bits = dataset.size
    return [(correct/total_bits * 100), higher_performing]

In [173]:
accuracy, higher_performing = predict(train_set)
print(f'\nAccuracy: {accuracy}')

Predicting: 100%|██████████| 549/549 [08:34<00:00,  1.07it/s]


Accuracy: 98.64837021007196


### 4. Determine your network’s accuracy on the test set. How well is the network performing?


In [172]:
accuracy, higher_performing = predict(test_set)
print(f'\nAccuracy: {accuracy}')

Predicting: 100%|██████████| 61/61 [00:58<00:00,  1.04it/s]


Accuracy: 98.45741440644026


In [153]:
train_set.size

5348358

In [154]:
row_train, col_train = train_set.shape
row_train * col_train

5348358

### 5. Choose a few higher-performing examples from the test set, then use movies.csv to determine which movies those users liked.
### Which other movies did the network predict that those users might like? Do the recommendations seem reasonable?


In [155]:
higher_performing

defaultdict(None,
            {0: array([-1, -1, -1, ..., -1, -1, -1]),
             1: array([ 1, -1, -1, ..., -1, -1, -1]),
             2: array([-1, -1, -1, ..., -1, -1, -1])})

In [188]:
pred = recall(W, higher_performing[0])
print(np.where(pred >= 0))

(array([277, 314]),)


In [189]:
higher_performing_0 = movies_df.iloc[np.where(higher_performing[0] >= 0)[0]]
higher_performing_0.to_csv('higher_performing[0].csv')
higher_performing_0

,movieId,title,genres
16,17,Sense and Sensibility (1995),Drama|Romance
31,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
43,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
97,110,Braveheart (1995),Action|Drama|War
123,150,Apollo 13 (1995),Adventure|Drama|IMAX
...,...,...,...
8569,116797,The Imitation Game (2014),Drama|Thriller|War
8592,117533,Citizenfour (2014),Documentary
8681,122882,Mad Max: Fury Road (2015),Action|Adventure|Sci-Fi|Thriller
8879,134130,The Martian (2015),Adventure|Drama|Sci-Fi


Which other movies did the network predict that those users might like? (first user)

In [190]:
# recommendation for the first higher_performing
movies_df.iloc[np.where(pred >= 0)[0]]

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War


In [191]:
# test for the second higher_performing
pred = recall(W, higher_performing[1])
print(np.where(pred >= 0))

(array([ 277,  314, 1734]),)


In [192]:
higher_performing_1 = movies_df.iloc[np.where(higher_performing[1] >= 0)[0]]
higher_performing_1.to_csv('higher_performing[1].csv')
higher_performing_1

,movieId,title,genres
9,10,GoldenEye (1995),Action|Adventure|Thriller
16,17,Sense and Sensibility (1995),Drama|Romance
20,21,Get Shorty (1995),Comedy|Crime|Thriller
33,36,Dead Man Walking (1995),Crime|Drama
43,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
...,...,...,...
6534,54286,"Bourne Ultimatum, The (2007)",Action|Crime|Thriller
6561,55052,Atonement (2007),Drama|Romance|War
6613,55820,No Country for Old Men (2007),Crime|Drama
6648,56782,There Will Be Blood (2007),Drama|Western


Which other movies did the network predict that those users might like? (second user)

In [193]:
# recommendation for higher_performing_1
movies_df.iloc[np.where(pred >= 0)[0]]

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War
1734,2329,American History X (1998),Crime|Drama


In [194]:
# test for the third higher_performing
pred = recall(W, higher_performing[2])
print(np.where(pred >= 0))

(array([277, 314]),)


In [195]:
higher_performing_2 = movies_df.iloc[np.where(higher_performing[2] >= 0)[0]]
higher_performing_2.to_csv('higher_performing[2].csv')
higher_performing_2

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
5,6,Heat (1995),Action|Crime|Thriller
9,10,GoldenEye (1995),Action|Adventure|Thriller
10,11,"American President, The (1995)",Comedy|Drama|Romance
16,17,Sense and Sensibility (1995),Drama|Romance
...,...,...,...
2315,3068,"Verdict, The (1982)",Drama|Mystery
2317,3071,Stand and Deliver (1988),Comedy|Drama
2318,3072,Moonstruck (1987),Comedy|Romance
3268,4424,"Garden of the Finzi-Continis, The (Giardino de...",Drama


In [196]:
# recommendation for higher_performing_1
movies_df.iloc[np.where(pred >= 0)[0]]

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War


Do the recommendations seem reasonable?
- Based on the recommendations, they seem reasonable because it is most likely that a lot of people have watched and rated those movies (e.g. Shawshank Redemption and Forrest Gump)

### 6. Construct a feature vector for yourself, choosing several movies that you’ve seen and liked. (Do not choose all movies that you have seen -- leave room for recommendations.) What other movies does the network recommend for you? How many of the recommended movies have you actually seen? Were the recommendations accurate?

In [162]:
# initialize array with value -1
my_test_sample = np.full((len(test_set[0]),), -1)
my_test_sample

array([-1, -1, -1, ..., -1, -1, -1])

In [163]:
my_movies = [1, 2, 362, 480, 1284, 6710, 7768]
for x in my_movies:
    my_test_sample[x-1] = 1

my_test_sample

array([ 1,  1, -1, ..., -1, -1, -1])

### What other movies does the network recommend for you?

In [164]:
my_pred = recall(W, my_test_sample)
movies_df.iloc[np.where(pred >= 0)[0]]

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War
788,1031,Bedknobs and Broomsticks (1971),Adventure|Children|Musical


### How many of the recommended movies have you actually seen?
None

### Were the recommendations accurate?
Yes, somewhat accurate because two of the movies refer to comedy and adventure genres which I actually like.